# Assignment 3: Evaluating Classification Models

#  Deilverables and Definition

# A.  Library and data import

In [1]:
import numpy as np
import pandas as pd
from math import sqrt  # for root mean-squared error calculation

#diplay and plotting
import seaborn as sns
from IPython.display import display
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import cm

#import from SKlearn
import sklearn.linear_model 
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error, r2_score  
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
import sklearn.metrics as metrics
from sklearn.pipeline import Pipeline

from scipy import stats

imported_data = pd.read_csv('C:/Users/bblank/Documents/Northwestern MSDS/datasets/bank.csv', sep=';')
imported_data.dropna() 
display(imported_data.shape)
imported_data.describe()
display(imported_data.head())

(4521, 17)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,response
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


In [2]:
#convert columns from y/n to 1/9
convert_to_binary = {'no' : 0, 'yes' : 1}

# define binary variable for having credit in default
default = imported_data['default'].map(convert_to_binary)
imported_data['default'] = default
# define binary variable for having a mortgage or housing loan
housing = imported_data['housing'].map(convert_to_binary)
imported_data['housing'] = housing
# define binary variable for having a personal loan
loan = imported_data['loan'].map(convert_to_binary)
imported_data['loan'] = loan
# define response variable to use in the model
response = imported_data['response'].map(convert_to_binary)
imported_data['response'] = response

display(imported_data.head())


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,response
0,30,unemployed,married,primary,0,1787,0,0,cellular,19,oct,79,1,-1,0,unknown,0
1,33,services,married,secondary,0,4789,1,1,cellular,11,may,220,1,339,4,failure,0
2,35,management,single,tertiary,0,1350,1,0,cellular,16,apr,185,1,330,1,failure,0
3,30,management,married,tertiary,0,1476,1,1,unknown,3,jun,199,4,-1,0,unknown,0
4,59,blue-collar,married,secondary,0,0,1,0,unknown,5,may,226,1,-1,0,unknown,0


In [3]:

df = imported_data.copy()
display(df.poutcome.value_counts())
display(df.contact.value_counts())
display(df.education.value_counts())
display(df.job.value_counts())
display(df.marital.value_counts())

unknown    3705
failure     490
other       197
success     129
Name: poutcome, dtype: int64

cellular     2896
unknown      1324
telephone     301
Name: contact, dtype: int64

secondary    2306
tertiary     1350
primary       678
unknown       187
Name: education, dtype: int64

management       969
blue-collar      946
technician       768
admin.           478
services         417
retired          230
self-employed    183
entrepreneur     168
unemployed       128
housemaid        112
student           84
unknown           38
Name: job, dtype: int64

married     2797
single      1196
divorced     528
Name: marital, dtype: int64

In [21]:
df.poutcome.value_counts()

unknown    3705
failure     490
other       197
success     129
Name: poutcome, dtype: int64

In [4]:
#poutcome != response
df[df.poutcome == 'success'].head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,response
49,61,admin.,married,unknown,0,4629,1,0,cellular,27,jan,181,1,92,1,success,1
70,37,management,married,tertiary,0,0,0,0,cellular,16,jul,268,2,182,3,success,1
98,36,blue-collar,divorced,secondary,0,2843,0,0,cellular,12,feb,473,1,182,1,success,0
110,21,student,single,secondary,0,2488,0,0,cellular,30,jun,258,6,169,3,success,1
156,55,admin.,married,secondary,0,602,0,0,cellular,1,jun,324,3,90,2,success,1
